# Data preprocessing

The code in this notebook takes the input pictures and applies the mediapipe hands solution to find the relevant coordinates and saves them as a .csv file which can be used by the AI model to train.

## Imports

In [ ]:
!pip install mediapipe

import pandas as pd
import cv2
import mediapipe as mp
import pandas as pd

## Download database

In [ ]:
!pip install kaggle
# kaggle API
!echo '{"username":"spacewaiker","key":"062bca10262b87f5b8498c589cdc65c6"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
# download dataset:
!kaggle datasets download grassknoted/asl-alphabet
# extract files (-q disables writing status message for every file)
!unzip asl-alphabet.zip -q
!ls

100% 1.02G/1.03G [00:16<00:00, 84.4MB/s]
100% 1.03G/1.03G [00:16<00:00, 64.8MB/s]


Symbols:

|Command  |ASL letters|Sign           |
|---------|-----------|---------------|
|Line     |L          |"L" shape      |
|Ellipse  |A, S       |Fist           |
|Rectangle|H, G       |Point right    |
|Triangle |K, V       |Two fingers up |
|Move     |B          |Palm           |
|Delete   |R          |Crossed fingers|

## Define functions to use later

In [100]:
'''
Function to return the file names with the following format:
A/A1234.jpg     if letter = 'A'  and   i = 1234
'''
def get_file_names(letter, n=3000):
    files = []
    for i in range(1, n+1):
        files.append(f'{letter}/{letter}{i}.jpg')
    
    return files

'''
Function that applies the mediapipe hands solution to an image.
img is the image file name and hands is the hands instance to use
'''
def apply_hands(row):
    global hands
    img = row.name
    result = row.copy()

    image = cv2.imread(DIR + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print(1, end='')
    
    landmk = hands.process(image).multi_hand_landmarks
    # print(2, end='')

    if landmk:    # success
        # print(3, end='')
        landmarks = landmk[0]
        # print(4, end='')
        result[1:22] = [(i.x, i.y, i.z) for i in landmarks.landmark]
        # print(5, end='')

    # print(6)
    return result


In [49]:
# Mediapipe stuff
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get file names
file_names = pd.DataFrame()

file_names = file_names.append([['Line', file] for file in get_file_names('L')])
file_names = file_names.append(
    [['Ellipse', file] for file in get_file_names('A', n=1500) + get_file_names('S', n=1500)])
file_names = file_names.append(
    [['Rectangle', file] for file in get_file_names('H', n=1500) + get_file_names('G', n=1500)])
file_names = file_names.append(
    [['Triangle', file] for file in get_file_names('K', n=1500) + get_file_names('V', n=1500)])
file_names = file_names.append([['Move', file] for file in get_file_names('B')])
file_names = file_names.append([['Delete', file] for file in get_file_names('R')])

file_names.rename(columns={0: 'Command', 1: 'File'}, inplace=True)
file_names.set_index('File', inplace=True)
print(file_names)

DIR = 'asl_alphabet_train/asl_alphabet_train/'

            Command
File               
L/L1.jpg       Line
L/L2.jpg       Line
L/L3.jpg       Line
L/L4.jpg       Line
L/L5.jpg       Line
...             ...
R/R2996.jpg  Delete
R/R2997.jpg  Delete
R/R2998.jpg  Delete
R/R2999.jpg  Delete
R/R3000.jpg  Delete

[18000 rows x 1 columns]


In [101]:
cols = ['Command'] + [i for i in range(21)]
train_x = pd.DataFrame(columns=cols, index=file_names.index)

train_x['Command'] = file_names['Command']


# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    train_x = train_x.apply(apply_hands, axis=1)  # 12 minutes

print(train_x)

            Command  ...                                                 20
File                 ...                                                   
L/L1.jpg       Line  ...  (0.1568724811077118, 0.6060709953308105, -0.03...
L/L2.jpg       Line  ...  (0.15856611728668213, 0.591741681098938, -0.06...
L/L3.jpg       Line  ...  (0.15946881473064423, 0.5749116539955139, -0.0...
L/L4.jpg       Line  ...  (0.16255448758602142, 0.5692355036735535, -0.1...
L/L5.jpg       Line  ...  (0.16627277433872223, 0.5448649525642395, -0.0...
...             ...  ...                                                ...
R/R2996.jpg  Delete  ...  (0.7475021481513977, 0.4340217113494873, -0.04...
R/R2997.jpg  Delete  ...  (0.7471187114715576, 0.44768959283828735, -0.0...
R/R2998.jpg  Delete  ...  (0.7431156635284424, 0.4530644416809082, -0.04...
R/R2999.jpg  Delete  ...  (0.7431281805038452, 0.4591083526611328, -0.04...
R/R3000.jpg  Delete  ...  (0.7425534725189209, 0.46314913034439087, -0.0...

[18000 rows

In [103]:
train_x.to_csv("drive/MyDrive/MAISFinalProject/train_x.csv")